In [14]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [15]:
import PrjtCF_module as cf

# Outline
* 호텔, 골프장 등 개발 후 운영 사업의 재무모델
* PF대출을 통해 토지비 및 공사비 등 개발사업 자금을 조달
* 금융비용은 PF대출금으로 충당
* 준공 후 담보대출금으로 상환 가정

# Input Index Data

In [16]:
# 기간 index 설정
idx = cf.PrjtIndex(idxname=['prjt', 'cstrn', 'loan'],
                   start=['2021-08', '2021-10', '2021-10'],
                   periods=[24+1, 18+1, 20+1],
                   freq='M')

# 공정률 index 설정
idx.prcs = Series(np.ones(len(idx.cstrn)) / len(idx.cstrn), 
                  index=idx.cstrn.index)

# Input Financing Data

In [17]:
equity = cf.Loan(idx, amt_ntnl=10_000)

loan = cf.Loan(idx.loan, amt_ntnl=60_000, rate_fee=0.02, 
               rate_IR=0.06)

# Input Cost Data

In [18]:
dct_cost = {}

# 토지비: 최초 1회 지급
lnd = cf.Account(idx)
lnd.addscdd(idx.cstrn[0], 10_000)
dct_cost['lnd'] = lnd

# 공사비: 공정률에 따라 지급
cstrn = cf.Account(idx)
cstrn.addscdd(idx.cstrn.index, 50_000 * idx.prcs)
dct_cost['cstrn'] = cstrn

cost = cf.Merge(dct_cost)

# Execution Cash Flow

In [19]:
# Make accounts : 운영현금흐름의 입출을 위한 운영계좌 개설
acc_oprtg = cf.Account(idx)

In [20]:
# Inflow of cash
def inflow_cash(idxno):
    equity.ntnl.send(idxno, equity.ntnl.sub_scdd[idxno], acc_oprtg)
    
    if idxno in idx.loan.index:
        loan.ntnl.send(idxno, loan.ntnl.sub_scdd[idxno], acc_oprtg)

In [21]:
# Outflow of cash
def outflow_cash(idxno):
    acc_oprtg.send(idxno, cost['lnd'].add_scdd[idxno], cost['lnd'])
    acc_oprtg.send(idxno, cost['cstrn'].add_scdd[idxno], cost['cstrn'])

In [22]:
# Pay financial cost
def pay_fnclcst(idxno):
    if idxno in idx.loan.index:
        acc_oprtg.send(idxno, loan.fee.add_scdd[idxno], loan.fee)
        acc_oprtg.send(idxno, loan.IR.add_scdd[idxno], loan.IR)

In [23]:
# Repay notional amount
def repay_ntnl(idxno):
    if idxno in idx.loan.index:
        acc_oprtg.send(idxno, loan.ntnl.add_scdd[idxno], loan.ntnl)

In [ ]:
# Execute cash flow
for idxno in idx.index:
    inflow_cash(idxno)
    outflow_cash(idxno)
    pay_fnclcst(idxno)
    repay_ntnl(idxno)

In [38]:
# Merge Accounts
dct_acc = {'equity':equity,
           'loan_ntnl':loan.ntnl,
           'loan_fee':loan.fee,
           'loan_IR':loan.IR,
           'cost_lnd':cost['lnd'],
           'cost_cstrn':cost['cstrn'],
           'acc_oprtg':acc_oprtg}
acc_merge = cf.Merge(dct_acc)

# Print Result

In [73]:
rslt_df = DataFrame({('equity.ntnl', 'amt_sub'):equity.ntnl.amt_sub[:],
                     ('loan.ntnl', 'amt_sub'):loan.ntnl.amt_sub[:],
                     ('loan.ntnl', 'amt_add'):loan.ntnl.amt_add[:],
                     ('loan.fee', 'amt_add'):loan.fee.amt_add[:],
                     ('loan.IR', 'amt_add'):loan.IR.amt_add[:],
                     ('cost_lnd', 'amt_add'):cost['lnd'].amt_add[:],
                     ('cost_cstrn', 'amt_add'):cost['cstrn'].amt_add[:],
                     ('acc_oprtg', 'amt_add'):acc_oprtg.amt_add[:],
                     ('acc_oprtg', 'amt_sub'):acc_oprtg.amt_sub[:],
                     ('acc_oprtg', 'bal_end'):acc_oprtg.bal_end[:]})

In [74]:
rslt_df.fillna(0)

equity.ntnl loan.ntnl          loan.fee loan.IR cost_lnd  \
               amt_sub   amt_sub  amt_add  amt_add amt_add  amt_add   
2021-08-31     10000.0       0.0      0.0      0.0     0.0      0.0   
2021-09-30         0.0       0.0      0.0      0.0     0.0      0.0   
2021-10-31         0.0   60000.0      0.0   1200.0     0.0  10000.0   
2021-11-30         0.0       0.0      0.0      0.0   300.0      0.0   
2021-12-31         0.0       0.0      0.0      0.0   300.0      0.0   
2022-01-31         0.0       0.0      0.0      0.0   300.0      0.0   
2022-02-28         0.0       0.0      0.0      0.0   300.0      0.0   
2022-03-31         0.0       0.0      0.0      0.0   300.0      0.0   
2022-04-30         0.0       0.0      0.0      0.0   300.0      0.0   
2022-05-31         0.0       0.0      0.0      0.0   300.0      0.0   
2022-06-30         0.0       0.0      0.0      0.0   300.0      0.0   
2022-07-31         0.0       0.0      0.0      0.0   300.0      0.0   
2022-08-31         0.0       0.0      0.0      0.0   300.0      0.0   
2022-09-30         0.0       0.0      0.0      0.0   300.0      0.0   
2022-10-31         0.0       0.0      0.0      0.0   300.0      0.0   
2022-11-30         0.0       0.0      0.0      0.0   300.0      0.0   
2022-12-31         0.0       0.0      0.0      0.0   300.0      0.0   
2023-01-31         0.0       0.0      0.0      0.0   300.0      0.0   
2023-02-28         0.0       0.0      0.0      0.0   300.0      0.0   
2023-03-31         0.0       0.0      0.0      0.0   300.0      0.0   
2023-04-30         0.0       0.0      0.0      0.0   300.0      0.0   
2023-05-31         0.0       0.0      0.0      0.0   300.0      0.0   
2023-06-30         0.0       0.0  60000.0      0.0   300.0      0.0   
2023-07-31         0.0       0.0      0.0      0.0     0.0      0.0   
2023-08-31         0.0       0.0      0.0      0.0     0.0      0.0   

             cost_cstrn acc_oprtg                              
                amt_add   amt_add       amt_sub       bal_end  
2021-08-31     0.000000   10000.0      0.000000  10000.000000  
2021-09-30     0.000000       0.0      0.000000  10000.000000  
2021-10-31  2631.578947   60000.0  13831.578947  56168.421053  
2021-11-30  2631.578947       0.0   2931.578947  53236.842105  
2021-12-31  2631.578947       0.0   2931.578947  50305.263158  
2022-01-31  2631.578947       0.0   2931.578947  47373.684211  
2022-02-28  2631.578947       0.0   2931.578947  44442.105263  
2022-03-31  2631.578947       0.0   2931.578947  41510.526316  
2022-04-30  2631.578947       0.0   2931.578947  38578.947368  
2022-05-31  2631.578947       0.0   2931.578947  35647.368421  
2022-06-30  2631.578947       0.0   2931.578947  32715.789474  
2022-07-31  2631.578947       0.0   2931.578947  29784.210526  
2022-08-31  2631.578947       0.0   2931.578947  26852.631579  
2022-09-30  2631.578947       0.0   2931.578947  23921.052632  
2022-10-31  2631.578947       0.0   2931.578947  20989.473684  
2022-11-30  2631.578947       0.0   2931.578947  18057.894737  
2022-12-31  2631.578947       0.0   2931.578947  15126.315789  
2023-01-31  2631.578947       0.0   2931.578947  12194.736842  
2023-02-28  2631.578947       0.0   2931.578947   9263.157895  
2023-03-31  2631.578947       0.0   2931.578947   6331.578947  
2023-04-30  2631.578947       0.0   2931.578947   3400.000000  
2023-05-31     0.000000       0.0    300.000000   3100.000000  
2023-06-30     0.000000       0.0  60300.000000 -57200.000000  
2023-07-31     0.000000       0.0      0.000000 -57200.000000  
2023-08-31     0.000000       0.0      0.000000 -57200.000000

In [64]:
cost.lnd

KeyError: 'lnd'